##### Import

In [1]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
%cd PROJECTDIRECTORY

In [3]:
#System imports
import sys
import glob
import os
import subprocess
import shutil
import datetime
from functools import reduce

#Data manipulation imports
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import math
import re
import itertools

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt

# Purpose

This notebook will synthesize data from different sources to prepare a data table for machine learning based on the development dictionary.

The data sources are:
- Development databook 2017-2021:
  - Annual data
  - per dev
- Asset Tracker
  - Fixed data
  - per dev
- weather.csv
  - Monthly data
  - every dev
- Waste Tonnage
  - Monthly data
  - per pickup consolidation

Steps:
1. Aggregate dev data to pickup consolidation
  - dev databook
  - asset tracker
2. Merge datasets, based on year, month, dsny name code




In [4]:
dev_dict = pd.read_excel('data/raw/devdict.xlsx')

## Step 1: Databook Cleaning

1. Read 17-21 databooks
2. Append dsny name code from dev dict
3. Data Cleaning
4. Aggregate to pickup consolidation
  - categorical data: senior development
  - numerical data

### Step 1: read 17-21 databooks

In [ ]:
df_17 = pd.read_csv('data/raw/db_2017.csv')
df_18 = pd.read_csv('data/raw/db_2017.csv')
df_19 = pd.read_csv('data/raw/db_2017.csv')
df_20 = pd.read_csv('data/raw/db_2017.csv')
df_21 = pd.read_csv('data/raw/db_2017.csv')

In [ ]:
df_17['year'] = 2017
df_18['year'] = 2018
df_19['year'] = 2019
df_20['year'] = 2020
df_21['year'] = 2021

In [ ]:
df_17.columns = ['hud_amp', 'tds', 'cons_tds', 'development_edp', 'operating_edp', 'hud',
'dev_name', 'program', 'method', 'type', 'section_8_transition_units',
'current_units', 'total_number_units', 'number_rental_rooms',
'avg._no._r/r_per_unit', 'population_-_section_8_transition',
'population_-_public_housing', 'total_population',
'residential_buildings', 'non-residential_buildings', 'stairhalls',
'stories', 'total_area_-_sq._ft.', 'acres', 'net_dev._area_-_sq._ft.',
'(excluding_park)_acres', 'bldg._coverage_-_sq._ft.',
'cubage_-_cu._ft.', 'bldg/land_coverage_-__%',
'density_(population/acre)', 'development_cost',
'cost_per_rental_room_(as_built)', 'avg._monthly_gross_rent',
'location', 'borough', 'community_district',
'us_congressional_district', 'new_york_state_senate_district',
'new_york_state_assembly_district', 'new_york_city_council_district',
'completion_date', 'federalized_development', 'senior_development',
'electricity_paid_by_residents', 'private_management', 'year']

In [ ]:
df_18.columns = ['hud_amp', 'tds', 'cons_tds', 'development_edp', 'operating_edp', 'hud',
'dev_name', 'program', 'method', 'type', 'section_8_transition_units',
'current_units', 'total_number_units', 'number_rental_rooms',
'avg._no._r/r_per_unit', 'population_-_section_8_transition',
'population_-_public_housing', 'total_population',
'residential_buildings', 'non-residential_buildings', 'stairhalls',
'stories', 'total_area_-_sq._ft.', 'acres', 'net_dev._area_-_sq._ft.',
'(excluding_park)_acres', 'bldg._coverage_-_sq._ft.',
'cubage_-_cu._ft.', 'bldg/land_coverage_-__%',
'density_(population/acre)', 'development_cost',
'cost_per_rental_room_(as_built)', 'avg._monthly_gross_rent',
'location', 'borough', 'community_district',
'us_congressional_district', 'new_york_state_senate_district',
'new_york_state_assembly_district', 'new_york_city_council_district',
'completion_date', 'federalized_development', 'senior_development',
'electricity_paid_by_residents', 'private_management', 'year']

In [ ]:
df_19.columns = ['hud_amp', 'tds', 'cons_tds', 'development_edp', 'operating_edp', 'hud',
'dev_name', 'program', 'method', 'type', 'section_8_transition_units',
'current_units', 'total_number_units', 'number_rental_rooms',
'avg._no._r/r_per_unit', 'population_-_section_8_transition',
'population_-_public_housing', 'total_population',
'families_on_fixed_income', '%_families_on_fixed_income',
'residential_buildings', 'non-residential_buildings', 'stairhalls',
'stories', 'total_area_-_sq._ft.', 'acres', 'net_dev._area_-_sq._ft.',
'(excluding_park)_acres', 'bldg._coverage_-_sq._ft.',
'cubage_-_cu._ft.', 'bldg/land_coverage_-__%',
'density_(population/acre)', 'development_cost',
'cost_per_rental_room_(as_built)', 'avg._monthly_gross_rent',
'location', 'borough', 'community_district',
'us_congressional_district', 'new_york_state_senate_district',
'new_york_state_assembly_district', 'new_york_city_council_district',
'completion_date', 'federalized_development', 'senior_development',
'electricity_paid_by_residents', 'private_management', 'year']

In [ ]:
df_20.columns = ['hud_amp', 'tds', 'cons_tds', 'development_edp', 'operating_edp', 'hud',
'dev_name', 'program', 'method', 'type', 'section_8_transition_units',
'current_units', 'total_number_units', 'number_rental_rooms',
'avg._no._r/r_per_unit', 'population_-_section_8_transition',
'population_-_public_housing', 'total_population',
'families_on_fixed_income', '%_families_on_fixed_income',
'residential_buildings', 'non-residential_buildings', 'stairhalls',
'stories', 'total_area_-_sq._ft.', 'acres', 'net_dev._area_-_sq._ft.',
'(excluding_park)_acres', 'bldg._coverage_-_sq._ft.',
'cubage_-_cu._ft.', 'bldg/land_coverage_-__%',
'density_(population/acre)', 'development_cost',
'cost_per_rental_room_(as_built)', 'avg._monthly_gross_rent',
'location', 'borough', 'community_district',
'us_congressional_district', 'new_york_state_senate_district',
'new_york_state_assembly_district', 'new_york_city_council_district',
'completion_date', 'federalized_development', 'senior_development',
'electricity_paid_by_residents', 'private_management', 'year']

In [ ]:
df = pd.concat([df_17, df_18, df_19, df_20, df_21])
df['tds'] = df['tds'].astype(str).str.zfill(3)

### Step 2: Append dsny name code from the dev dict

In [ ]:
dev = dev_dict[['tds', 'dev_name_tracker','dsny_namecode',  'namecode_tds']]

In [ ]:
dev['tds'] = dev['tds'].astype(str).str.zfill(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df = df.merge(dev, on = 'tds')
df = df[~df.dsny_namecode.isna()]

### Step 3: Data cleaning

1. Categorize the columns
2. Fill `NaN`
3. Convert text format to numeric

In [ ]:
df = df.rename(columns = {"avg._no._r/r_per_unit":'ave_r_per_unit',
                          "total_area_-_sq._ft.":'total_sqft',
                          'net_dev._area_-_sq._ft.':'net_dev_sqft',
                          '(excluding_park)_acres':'acres_nopark',
                          'bldg._coverage_-_sq._ft.':'bldg_coverage_sqft',
                          'cubage_-_cu._ft.':'cubage_cuft',
                          'bldg/land_coverage_-__%': 'bldg_land_ratio',
                          '%_families_on_fixed_income':'pct_families_on_fixed_income',
                          'avg._monthly_gross_rent':'avg_m_gross_rent',
                          'cost_per_rental_room_(as_built)':'cost_per_rental_room_ab',
                          'population_-_public_housing':'population_PH',
                          'population_-_section_8_transition':'population_S8',
                          'total_population':'population_total',
                          'density_(population/acre)':'density_PopPerAcre',
                          'section_8_transition_units':'units_S8',
                          'current_units':'units_current',
                          'total_number_units':'units_total'})

In [ ]:
id_col = ['hud_amp', 'tds', 'cons_tds', 'development_edp', 'operating_edp', 'hud',
       'dev_name','location', 'borough', 'community_district',
       'us_congressional_district', 'new_york_state_senate_district',
       'new_york_state_assembly_district', 'new_york_city_council_district']

cate_col = ['program', 'method', 'type','senior_development',
       'electricity_paid_by_residents','private_management']

units_col = ['units_S8', 'units_current', 'units_total', 'number_rental_rooms']

pop_col = ['ave_r_per_unit', 'population_S8',
       'population_PH', 'population_total', 'density_PopPerAcre']

builtenv_col = ['residential_buildings', 'non-residential_buildings', 'stairhalls',
                'total_sqft', 'acres', 'net_dev_sqft',
                'acres_nopark', 'bldg_coverage_sqft',
                'cubage_cuft', 'bldg_land_ratio']

socio_col = ['development_cost','cost_per_rental_room_ab', 
             'avg_m_gross_rent', 'families_on_fixed_income', 
             'pct_families_on_fixed_income']

In [ ]:
df['private_management'] = df['private_management'].fillna('NO')
df['electricity_paid_by_residents'] = df['electricity_paid_by_residents'].fillna('NO')
df['senior_development'] = df['senior_development'].fillna('NO')

In [ ]:
for i in ['tds', 'cons_tds', 'operating_edp', 'us_congressional_district', 'year', 'namecode_tds']:
  df[i] = df[i].astype(str)

df['tds'] = df['tds'].str.zfill(3)
df['cons_tds'] = df['cons_tds'].str.zfill(3)
df['namecode_tds'] = df['namecode_tds'].str.zfill(3)

In [ ]:
for i in (units_col+pop_col+builtenv_col+socio_col):
  try:
    df[i] = df[i].str.replace('$', '')
  except:
    try:
      df[i] = df[i].astype(float)
    except:
      pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
for i in (units_col+pop_col+builtenv_col+socio_col):
  try:
    df[i] = df[i].str.replace('%', '')
  except:
    try:
      df[i] = df[i].astype(float)
    except:
      pass

In [ ]:
for i in (units_col+pop_col+builtenv_col+socio_col):
  try:
    df[i] = df[i].str.replace(',', '').astype(float)
  except:
    try:
      df[i] = df[i].astype(float)
    except:
      pass

In [ ]:
#df.to_csv('/gdrive/MyDrive/000 Spring 2022/Thesis Capstone/Data/Interim/1721db_dsnynamecode.csv', index = False)

### Step 4: Aggregate to consolidation by dsny code

1. Categorical data
  - Senior Development: binary

2. Numerical data
  - summing 
  - average

#### All Column Categorization

In [ ]:
#df = pd.read_csv('/gdrive/MyDrive/000 Spring 2022/Thesis Capstone/Data/Interim/1721db_dsnynamecode.csv')

In [ ]:
df[df.dsny_namecode == 'sotomayor']

,hud_amp,tds,cons_tds,development_edp,operating_edp,hud,dev_name,program,method,type,...,federalized_development,senior_development,electricity_paid_by_residents,private_management,year,families_on_fixed_income,pct_families_on_fixed_income,dev_name_tracker,dsny_namecode,namecode_tds
225,NY005010670,067,067,222,222,NY005022,SOTOMAYOR HOUSES,FEDERAL,CONVENTIONAL,NEW CONST,...,NaN,NO,NO,NO,2017,NaN,NaN,SOTOMAYOR HOUSES,sotomayor,067
226,NY005010670,067,067,222,222,NY005022,SOTOMAYOR HOUSES,FEDERAL,CONVENTIONAL,NEW CONST,...,NaN,NO,NO,NO,2018,NaN,NaN,SOTOMAYOR HOUSES,sotomayor,067
227,NY005010670,067,067,222,222,NY005022,SOTOMAYOR HOUSES,FEDERAL,CONVENTIONAL,NEW CONST,...,NaN,NO,NO,NO,2019,NaN,38.40,SOTOMAYOR HOUSES,sotomayor,067
228,NY005010670,067,067,222,222,NY005022,SOTOMAYOR HOUSES,FEDERAL,CONVENTIONAL,NEW CONST,...,NaN,NO,NO,NO,2020,NaN,39.30,SOTOMAYOR HOUSES,sotomayor,067
229,NY005010670,067,067,222,222,NY005022,SOTOMAYOR HOUSES,FEDERAL,CONVENTIONAL,NEW CONST,...,NaN,NO,NO,NO,2021,585.0,39.69,SOTOMAYOR HOUSES,sotomayor,067


In [ ]:
id_col = ['hud_amp', 'tds', 'cons_tds', 'development_edp', 'operating_edp', 'hud',
       'dev_name','location', 'borough', 'community_district',
       'us_congressional_district', 'new_york_state_senate_district',
       'new_york_state_assembly_district', 'new_york_city_council_district']

cate_col = ['program', 'method', 'type','senior_development',
       'electricity_paid_by_residents','private_management']

units_col = ['units_S8', 'units_current', 'units_total', 'number_rental_rooms']

pop_col = ['population_S8','population_PH', 'population_total']

builtenv_col = ['residential_buildings', 'non-residential_buildings', 'stairhalls',
                'total_sqft', 'acres', 'net_dev_sqft',
                'acres_nopark', 'bldg_coverage_sqft',
                'cubage_cuft']

socio_col = ['development_cost', 'families_on_fixed_income']

density = ['ave_r_per_unit', 'density_PopPerAcre']
bldg_land = ['bldg_land_ratio']
unit_rent = ['avg_m_gross_rent']
unit_cost = ['cost_per_rental_room_ab']
pct_FI_families = ['pct_families_on_fixed_income']

In [ ]:
for col in cate_col:
  print(col)
  print(pd.unique(df[col]))
  print('-----------------------------------------')

program
['FEDERAL' 'MIXED FINANCE/LLC1' 'MIXED FINANCE/LLC2']
-----------------------------------------
method
['CONVENTIONAL' 'TURNKEY']
-----------------------------------------
type
['NEW CONST' 'REHAB' 'NEW CONST (ELD)' 'REHAB (ELD)']
-----------------------------------------
senior_development
['NO' 'EXCLUSIVELY' 'PARTIALLY' 'PARTIALLY (1 BUILDING)'
 'PARTIALLY (1 STAIRHALL)' 'PARTIALLY (2 BUILDINGS)']
-----------------------------------------
electricity_paid_by_residents
['NO' 'YES']
-----------------------------------------
private_management
['NO']
-----------------------------------------


#### Senior Development: Binary

In [ ]:
df.loc[df['senior_development']!='NO','SD'] = 1
df.loc[df['senior_development']=='NO','SD'] = 0

In [ ]:
df_full = df[['year','dsny_namecode', 'namecode_tds','SD']].groupby(['year','dsny_namecode', 'namecode_tds']).mean().reset_index()

In [ ]:
df_full.loc[df_full['SD'] >0, 'SD'] =1

In [ ]:
df_full['SD'] = df_full['SD'].astype(int).astype(str)

In [ ]:
df_full[df_full.dsny_namecode == 'sotomayor']

,year,dsny_namecode,namecode_tds,SD
103,2017,sotomayor,067,0
230,2018,sotomayor,067,0
357,2019,sotomayor,067,0
484,2020,sotomayor,067,0
611,2021,sotomayor,067,0


#### Numerical Variables

Summation

Additional Calculation

Mean

In [ ]:
df.columns

Index(['hud_amp', 'tds', 'cons_tds', 'development_edp', 'operating_edp', 'hud',
       'dev_name', 'program', 'method', 'type', 'units_S8', 'units_current',
       'units_total', 'number_rental_rooms', 'ave_r_per_unit', 'population_S8',
       'population_PH', 'population_total', 'residential_buildings',
       'non-residential_buildings', 'stairhalls', 'stories', 'total_sqft',
       'acres', 'net_dev_sqft', 'acres_nopark', 'bldg_coverage_sqft',
       'cubage_cuft', 'bldg_land_ratio', 'density_PopPerAcre',
       'development_cost', 'cost_per_rental_room_ab', 'avg_m_gross_rent',
       'location', 'borough', 'community_district',
       'us_congressional_district', 'new_york_state_senate_district',
       'new_york_state_assembly_district', 'new_york_city_council_district',
       'completion_date', 'federalized_development', 'senior_development',
       'electricity_paid_by_residents', 'private_management', 'year',
       'families_on_fixed_income', 'pct_families_on_fixed_income',
 

In [ ]:
cols_sum = units_col + pop_col + builtenv_col + socio_col + ['year', 'dsny_namecode']
df_sum = df[cols_sum].groupby(['year', 'dsny_namecode']).sum().reset_index()

In [ ]:
df_full = df_full.merge(df_sum, on = ['year', 'dsny_namecode'])

In [ ]:
df_full['ave_r_per_unit'] = df_full['population_total']/df_full['units_total']
df_full['density_PopPerSqFt'] = df_full['population_total']/df_full['total_sqft']
df_full['density_PopPerDevFt'] = df_full['population_total']/df_full['net_dev_sqft']
df_full['density_PopPerCubage'] = df_full['population_total']/df_full['cubage_cuft']

In [ ]:
df_full['cost_per_rental_room_ab'] = df_full['development_cost']/df_full['units_total']

In [ ]:
bldg_land = ['bldg_land_ratio']
unit_rent = ['avg_m_gross_rent']
pct_FI_families = ['pct_families_on_fixed_income']

In [ ]:
cols_mean = bldg_land + unit_rent + pct_FI_families + ['year', 'dsny_namecode']
df_mean = df[cols_mean].groupby(['year', 'dsny_namecode']).mean().reset_index()

In [ ]:
df_full = df_full.merge(df_mean, on = ['year', 'dsny_namecode'])

In [ ]:
df_full[df_full.dsny_namecode == 'sotomayor']

,year,dsny_namecode,namecode_tds,SD,units_S8,units_current,units_total,number_rental_rooms,population_S8,population_PH,...,development_cost,families_on_fixed_income,ave_r_per_unit,density_PopPerSqFt,density_PopPerDevFt,density_PopPerCubage,cost_per_rental_room_ab,bldg_land_ratio,avg_m_gross_rent,pct_families_on_fixed_income
103,2017,sotomayor,067,0,0.0,1497.0,1497.0,6971.5,0.0,3375.0,...,17963549.0,0.0,2.254509,0.002518,0.002518,0.000276,11999.698731,14.21,514.0,NaN
230,2018,sotomayor,067,0,0.0,1496.0,1497.0,6968.0,0.0,3364.0,...,17963549.0,0.0,2.247161,0.002509,0.002509,0.000275,11999.698731,14.21,522.0,NaN
357,2019,sotomayor,067,0,0.0,1495.0,1497.0,6963.5,0.0,3286.0,...,17963549.0,0.0,2.195057,0.002451,0.002451,0.000269,11999.698731,14.21,542.0,38.40
484,2020,sotomayor,067,0,0.0,1496.0,1497.0,6968.0,0.0,3178.0,...,17963549.0,0.0,2.122912,0.002371,0.002371,0.000260,11999.698731,14.21,544.0,39.30
611,2021,sotomayor,067,0,0.0,1495.0,1497.0,6963.5,0.0,3167.0,...,17963549.0,585.0,2.115564,0.002363,0.002363,0.000259,11999.698731,14.21,542.0,39.69


##### dev names

In [ ]:
df_name = df[['dev_name', 'dsny_namecode']].drop_duplicates().groupby('dsny_namecode').agg({'dev_name':', '.join}).reset_index()

In [ ]:
df_name.head(10)

,dsny_namecode,dev_name
0,adams,ADAMS
1,albany,"ALBANY, ALBANY II"
2,amsterdam,"AMSTERDAM, AMSTERDAM ADDITION"
3,astoria,ASTORIA
4,atlanticterminal,ATLANTIC TERMINAL SITE 4B
5,audubon,AUDUBON
6,baileyavenuewest193rdstreet,BAILEY AVENUE-WEST 193RD STREET
7,baisleypark,BAISLEY PARK
8,baruch,"BARUCH, BARUCH HOUSES ADDITION"
9,bayview,BAY VIEW


In [ ]:
df_full = df_full.merge(df_name, on = ['dsny_namecode'])

In [ ]:
df_full[df_full.dsny_namecode == 'sotomayor']

,year,dsny_namecode,namecode_tds,SD,units_S8,units_current,units_total,number_rental_rooms,population_S8,population_PH,...,families_on_fixed_income,ave_r_per_unit,density_PopPerSqFt,density_PopPerDevFt,density_PopPerCubage,cost_per_rental_room_ab,bldg_land_ratio,avg_m_gross_rent,pct_families_on_fixed_income,dev_name
515,2017,sotomayor,067,0,0.0,1497.0,1497.0,6971.5,0.0,3375.0,...,0.0,2.254509,0.002518,0.002518,0.000276,11999.698731,14.21,514.0,NaN,SOTOMAYOR HOUSES
516,2018,sotomayor,067,0,0.0,1496.0,1497.0,6968.0,0.0,3364.0,...,0.0,2.247161,0.002509,0.002509,0.000275,11999.698731,14.21,522.0,NaN,SOTOMAYOR HOUSES
517,2019,sotomayor,067,0,0.0,1495.0,1497.0,6963.5,0.0,3286.0,...,0.0,2.195057,0.002451,0.002451,0.000269,11999.698731,14.21,542.0,38.40,SOTOMAYOR HOUSES
518,2020,sotomayor,067,0,0.0,1496.0,1497.0,6968.0,0.0,3178.0,...,0.0,2.122912,0.002371,0.002371,0.000260,11999.698731,14.21,544.0,39.30,SOTOMAYOR HOUSES
519,2021,sotomayor,067,0,0.0,1495.0,1497.0,6963.5,0.0,3167.0,...,585.0,2.115564,0.002363,0.002363,0.000259,11999.698731,14.21,542.0,39.69,SOTOMAYOR HOUSES


## Step 2: Asset Tracker Cleaning

1. Read the Asset Tracker
2. Append dsny name code from dev dict
3. Data Cleaning
4. Aggregate to pickup consolidation

In [ ]:
df_extcomp = pd.read_excel('/data/raw/CorrectionTrackers_PreCleaning.xlsx', sheet_name='extcomp')
df_intcomp = pd.read_excel('/data/raw/CorrectionTrackers_PreCleaning.xlsx', sheet_name='intcomp')
df_cont = pd.read_excel('/data/raw/CorrectionTrackers_PreCleaning.xlsx', sheet_name='cont')

In [ ]:
df_extcomp = df_extcomp[[' Development TDS #', 'Development Name', 'New Number of Exterior Compactors','New Number of Waste Yards']]
df_extcomp.columns = ['tds', 'dev_name', 'extcomp', 'waste_yard']
df_extcomp = df_extcomp[~df_extcomp.tds.isna()]

In [ ]:
df_intcomp = df_intcomp[[' Development TDS #', 'Development Name', 'New Number of Interior Compactors ']]
df_intcomp.columns = ['tds', 'dev_name', 'intcomp']

In [ ]:
df_cont = df_cont[[' Development TDS #', 'Development Name', 'New Number of Recycling Bins','New Number of cardboard Balers','New Number of Bulk Containers']]
df_cont.columns = ['tds', 'dev_name', 'rec_bins', 'balers', 'bulk_conts']

In [ ]:
df_extcomp.tds = df_extcomp.tds.astype(int)
df_intcomp.tds = df_intcomp.tds.astype(int)
df_cont.tds = df_cont.tds.astype(int)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df_tracker = dev_dict[['tds', 'dsny_namecode', 'namecode_tds']].merge(df_extcomp, on = 'tds', how = 'left')
df_tracker = df_tracker.merge(df_intcomp[['tds', 'intcomp']], on = 'tds')
df_tracker = df_tracker.merge(df_cont[['tds','rec_bins', 'balers', 'bulk_conts']], on = 'tds')

In [ ]:
df_tracker = df_tracker.drop(columns='tds')

In [ ]:
for i in [ 'extcomp','waste_yard', 'intcomp', 'rec_bins', 'balers', 'bulk_conts']:
  df_tracker[i] = df_tracker[i].astype(float)

In [ ]:
df_tracker = df_tracker.groupby(['dsny_namecode', 'namecode_tds']).sum().reset_index()

In [ ]:
df_tracker['namecode_tds'] = df_tracker['namecode_tds'].astype(str).str.zfill(3)
df_full['namecode_tds'] = df_full['namecode_tds'].astype(str).str.zfill(3)

In [ ]:
df_full = df_full.merge(df_tracker, on = ['dsny_namecode', 'namecode_tds'])

##### Per units

In [ ]:
df_full['per_unit_rb'] = df_full['rec_bins']/df_full['units_total']
df_full['per_capita_rb'] = df_full['rec_bins']/df_full['population_total']

## Step 3: Integrate Waste Tonnage Data

1. Read the tonnage and the rolling tonnage
2. Merge dataframes

In [ ]:
df_raw_ton = pd.read_csv('data/raw/1721_devcode_ton_cont_month.csv')
df_raw_ton = df_raw_ton.rename(columns={'name_code':'dsny_namecode'})

In [ ]:
change_dict = {'stebbinshewitt': 'davidson', 'bronxdale':'sotomayor'}

df_raw_ton['dsny_namecode'] = df_raw_ton['dsny_namecode'].replace(change_dict)
df_raw_ton = df_raw_ton.groupby(['dsny_namecode', 'month', 'year', 'time']).sum().reset_index()

In [ ]:
df_raw_ton.year = df_raw_ton.year.astype(str)

In [ ]:
df_full = df_raw_ton[['dsny_namecode', 'year', 'time', 'cont', 'ton']].merge(df_full, on = ['dsny_namecode', 'year'])

## Step 4: Weather Cleaning

1. Read the weather data
2. append to df_full

In [ ]:
df_weather = pd.read_csv('data/raw/weather.csv')

In [ ]:
df_weather['year'] = pd.to_datetime(df_weather['year'])
df_weather = df_weather.rename(columns={'year':'time'})
df_full['time'] = pd.to_datetime(df_full['time'])

In [ ]:
df_full = df_full.merge(df_weather[['time', 'Avg Temperature', 'Precipitation', 'Snowdepth']], on = 'time')

## Step 5: Cluster Label

In [ ]:
df_labels = pd.read_excel('data/raw/clusterlabels.xlsx')

In [ ]:
df_labels = df_labels.rename(columns={'index':'dsny_namecode'})

In [ ]:
change_dict = {'bronxdale':'sotomayor'}
df_labels['dsny_namecode'] = df_labels['dsny_namecode'].replace(change_dict)

In [ ]:
df_labels = df_labels[df_labels['dsny_namecode']!='stebbinshewitt']

In [ ]:
df_full = df_full.merge(df_labels[['dsny_namecode', 'label_n4']], on = 'dsny_namecode')

## Step 6: Additional Features

In [ ]:
# One Hot encoding for clusters: label_n4
from sklearn.preprocessing import OneHotEncoder
df_full.loc[df_full.label_n4 == 0, 'label_n4'] = 'C1'
df_full.loc[df_full.label_n4 == 1, 'label_n4'] = 'C2'
df_full.loc[df_full.label_n4 == 2, 'label_n4'] = 'C3'
df_full.loc[df_full.label_n4 == 3, 'label_n4'] = 'C4'
df_full.loc[df_full.label_n4 == -1, 'label_n4'] = 'O'
one_hot = pd.get_dummies(df_full['label_n4'])
df_full = df_full.join(one_hot[['C1', 'C2', 'C3', 'C4']])

In [ ]:
# Precipitation
rain_data = df_full[['time', 'Precipitation']].drop_duplicates()['Precipitation']

mean= np.mean(rain_data)
standardDeviation = np.std(rain_data)
breaks = [rain_data.min(), mean-standardDeviation, mean, mean+standardDeviation, mean+standardDeviation*2, rain_data.max()]

rain_data = pd.DataFrame(rain_data)
stDevBreakLabels = ['< -1 StDev', '-1 - 0 StDev', '0 - +1 StDev', '+1 - +2 StDev', '> +2 StDev']
rain_data['stDev'] = pd.cut(rain_data['Precipitation'], breaks, labels=stDevBreakLabels)
rain_data.groupby('stDev').count()

# Define larger than Mean+1STD as heavy precipitation
df_full.loc[df_full.Precipitation >0.198, 'HeavyPre'] = 1
df_full.loc[df_full.Precipitation <= 0.198, 'HeavyPre'] = 0

In [ ]:
# Snow
df_full.loc[df_full.Snowdepth >0, 'Snow'] = 1
df_full.loc[df_full.Snowdepth == 0, 'Snow'] = 0

In [ ]:
# Month
df_full.time = pd.to_datetime(df_full.time)
df_full['month'] = df_full.time.dt.month_name()

one_hot = pd.get_dummies(df_full['month'])
df_full = df_full.join(one_hot[['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']])

In [ ]:
# Pandemic
df_full.loc[df_full.time>'2020-03-01', 'pandemic'] = 1
df_full.loc[df_full.time<'2020-04-01', 'pandemic'] = 0

In [ ]:
# none development square feet
df_full['nondev_sqft'] = df_full['total_sqft'] - df_full['net_dev_sqft']
df_full['park_acres'] = df_full['acres'] - df_full['acres_nopark']

## Step 7: OCT NOV 2021 Waste Tonnage Data

In [ ]:
df_moredata = pd.read_excel('data/raw/data_2021_Oct_Nov.xlsx')

In [ ]:
df_moredata = df_moredata[['dsny_namecode', 'Oct', 'Nov']]
df_moredata = df_moredata.dropna()

In [ ]:
df_moredata = df_moredata.rename(columns={'Oct':'year10', 'Nov':'year11'})

In [ ]:
df_wide = pd.wide_to_long(df_moredata, stubnames='year', i=['dsny_namecode'], j='month').reset_index()
df_wide = df_wide.rename(columns={'year':'ton_1'})

In [ ]:
df_wide['month'] = df_wide['month'].astype(str)
df_wide['time'] = '2021-'+df_wide['month']+'-01'
df_wide['time'] = pd.to_datetime(df_wide['time'])

In [ ]:
df_full = df_full.merge(df_wide[['dsny_namecode','time', 'ton_1']], on = ['dsny_namecode','time'], how = 'left')
df_full.loc[~df_full.ton_1.isna(), 'ton'] = df_full['ton_1']

df_full = df_full.drop(columns = 'ton_1')

## Step 8: Impute data and Write csv

In [ ]:
df_full = df_full[df_full.time != '2021-12-01']

In [ ]:
df_impute = df_full.groupby(['dsny_namecode', 'year']).agg({'ton':'mean'}).reset_index()
df_impute = df_impute.rename(columns = {'ton':'mean_ton'})

In [ ]:
df_full = df_full.merge(df_impute, on = ['dsny_namecode', 'year'], how = 'left')
df_full.loc[df_full.ton ==0, 'ton'] = df_full['mean_ton']

In [ ]:
df_full = df_full.sort_values(by = ['dsny_namecode','time']).reset_index(drop = True)
df_full['rolling_ton'] = df_full.groupby('dsny_namecode')['ton'].rolling(3).mean().shift(-1).reset_index()['ton']

In [ ]:
df_full.to_csv('data/fulldata_impute.csv', index = False)